In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', '{:,.4f}'.format)

# <font color='navy'> Datos

In [2]:
df = pd.read_csv('losses_ex.csv')
ecl =  5_946_400
std =  4_475_800
var =  20_996_000 
cvar =  23_818_000

#ecl = 392_510
#std = 204_580
#var = 989_700
#cvar = 1_101_200 

ucl = var - ecl

# <font color='navy'> Tranches

In [3]:
port = 118_553_188
equity = ecl * 0.6
mezanine = ecl * 0.80
junior = ecl * 1.1

#port = 5_552_048
#equity = ecl
#mezanine = ecl * 1.2
#junior = ecl * 1.3

senior = port - ecl - equity - mezanine - junior

In [4]:
lim_ecl = ecl
lim_equity = ecl + equity
lim_mezanine = ecl + equity + mezanine
lim_junior = ecl + equity + mezanine + junior
lim_senior = ecl + equity + mezanine + junior + senior

# <font color='navy'> Grupos

In [5]:
maximo = df.max().values[0]
minimo = df.min().values[0]
rango = maximo - minimo

In [6]:
df_grupos = pd.DataFrame({
    'grupos': list(range(1, 101)),
})
df_grupos['perdida'] = minimo + rango/99 * df_grupos.index

In [7]:
losses = df['losses'].to_numpy()
bins = df_grupos['perdida'].to_numpy()

frecuencias = []

for i, limite in enumerate(bins):
    if i == 0:
        count = np.sum(losses <= limite)
    else:
        count = np.sum((losses > bins[i-1]) & (losses <= limite))
    frecuencias.append(count)

df_grupos['frecuencia'] = frecuencias
df_grupos['prob_marginal'] = df_grupos['frecuencia'] / df_grupos['frecuencia'].sum()
df_grupos['prob_acumulada'] = df_grupos['prob_marginal'].cumsum()
df_grupos

,grupos,perdida,frecuencia,prob_marginal,prob_acumulada
0,1,0.0000,11,0.0000,0.0000
1,2,"374,605.0330",2725,0.0027,0.0027
2,3,"749,210.0661",15137,0.0151,0.0179
3,4,"1,123,815.0991",30622,0.0306,0.0485
4,5,"1,498,420.1321",42469,0.0425,0.0910
...,...,...,...,...,...
95,96,"35,587,478.1379",4,0.0000,1.0000
96,97,"35,962,083.1709",0,0.0000,1.0000
97,98,"36,336,688.2039",3,0.0000,1.0000
98,99,"36,711,293.2370",2,0.0000,1.0000


# <font color='navy'> Cortes

In [8]:
df_filtered = df_grupos[df_grupos['perdida'] < lim_ecl]
df_extra = df_grupos[df_grupos['perdida'] >= lim_ecl].head(1)
df_ecl = pd.concat([df_filtered, df_extra])

df_filtered_equity = df_grupos[(df_grupos['perdida'] > df_ecl['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_equity)]
df_extra_equity = df_grupos[df_grupos['perdida'] >= lim_equity].head(1)
df_equity = pd.concat([df_filtered_equity, df_extra_equity])

df_filtered_mezanine = df_grupos[(df_grupos['perdida'] > df_equity['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_mezanine)]
df_extra_mezanine = df_grupos[df_grupos['perdida'] >= lim_mezanine].head(1)
df_mezanine = pd.concat([df_filtered_mezanine, df_extra_mezanine])

df_filtered_junior = df_grupos[(df_grupos['perdida'] > df_mezanine['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_junior)]
df_extra_junior = df_grupos[df_grupos['perdida'] >= lim_junior].head(1)
df_junior = pd.concat([df_filtered_junior, df_extra_junior])

df_filtered_senior = df_grupos[(df_grupos['perdida'] > df_junior['perdida'].iloc[-1]) & (df_grupos['perdida'] < lim_senior)]
df_extra_senior = df_grupos[df_grupos['perdida'] >= lim_senior].head(1)
df_senior = pd.concat([df_filtered_senior, df_extra_senior])

# <font color='navy'> Métricas

In [9]:
equity_loss_prob = 1 - df_equity['prob_acumulada'].iloc[0]
mezanine_loss_prob = 1 - df_mezanine['prob_acumulada'].iloc[0]
junior_loss_prob = 1 - df_junior['prob_acumulada'].iloc[0]

if len(df_senior) > 0:
    senior_loss_prob = 1 - df_senior['prob_acumulada'].iloc[0]
else:
    senior_loss_prob = 0

In [10]:
equity_loss = np.sum((df_equity['perdida'] - df_ecl['perdida'].iloc[-1]) * df_equity['prob_marginal'])
mezanine_loss = np.sum((df_mezanine['perdida'] - df_equity['perdida'].iloc[-1]) * df_mezanine['prob_marginal'])
junior_loss = np.sum((df_junior['perdida'] - df_mezanine['perdida'].iloc[-1]) * df_junior['prob_marginal'])
senior_loss = np.sum((df_senior['perdida'] - df_junior['perdida'].iloc[-1]) * df_senior['prob_marginal'])

In [11]:
expected_loss = np.array([equity_loss, mezanine_loss, junior_loss, senior_loss])
port_values = np.array([equity, mezanine, junior, senior])
spread = expected_loss / port_values

In [12]:
df_resultados = pd.DataFrame({
    'tramo': ['equity', 'mezanine', 'junior', 'senior'],
    'probabilidad': [equity_loss_prob, mezanine_loss_prob, junior_loss_prob, senior_loss_prob],
    'pérdida esperada': expected_loss,
    'spread': spread
})
df_resultados['probabilidad'] = df_resultados['probabilidad'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados['spread'] = df_resultados['spread'].apply(lambda x: '{:,.2%}'.format(x))
df_resultados

,tramo,probabilidad,pérdida esperada,spread
0,equity,35.30%,"385,879.5207",10.82%
1,mezanine,15.73%,"255,209.0439",5.36%
2,junior,5.09%,"119,328.1856",1.82%
3,senior,0.90%,"30,325.4012",0.03%
